In [ ]:
from cv2 import cv2
import numpy as np
import os

# Playing video from file:
cap = cv2.VideoCapture('input.mp4') # change this to video in question 

try:
    if not os.path.exists('data'):
        os.makedirs('data')
except OSError:
    print ('Error: Creating directory of data')

currentFrame = 0
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Saves image of the current frame in jpg file
    name = './data/' + str(int(currentFrame)) + '.jpg'
    print ('Creating...' + name)
    cv2.imwrite(name, frame)

    # To stop duplicate images
    currentFrame += 1

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [33]:
import requests
import json

predictionUrl = 'https://myazurecustomvision.cognitiveservices.azure.com/customvision/v3.0/Prediction/fd698d74-978e-440e-b34f-a802c01f5d87/detect/iterations/Iteration3/image'
predictionKey = '9a1d0210845f417c8728965579867416'

def predict(image):
  headers = {'content-type' : 'application/octet-stream','Prediction-Key' : predictionKey}
  request = requests.post(predictionUrl,data=open(image,"rb"),headers=headers)

  data = json.loads(request.content.decode('utf8').replace("'", '"'))['predictions']

  predictions = []

  for i in data:
    if i['probability'] > 0.7:
      predictions.append([i['tagName'], i['boundingBox']])

  return predictions

In [34]:
import time

myList = []

for i in range(0, len(os.listdir('data')) - 1):
  myList.append(predict("./data/" + str(i) + ".jpg"))
  time.sleep(0.5)

In [35]:
def drawBox(boxDetails, image, i):
  for j in boxDetails:
    x = j[1]['left']
    y = j[1]['top']
    w = j[1]['width']
    h = j[1]['height']
    
    width, height, channels = image.shape

    startX = int(x * height)
    startY = int(y * width)
    endX = int(w * height) + startX
    endY = int(h * width) + startY

    cv2.putText(image, j[0], (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (155, 255, 0), 1)
    cv2.rectangle(image, (startX, startY), (endX, endY), (155, 255, 0), 2)
  cv2.imwrite("data/%d.jpg" % i, image)

In [36]:
for i in range(0, len(os.listdir('data')) - 1):
  image = cv2.imread("./data/" + str(i) + ".jpg")
  drawBox(myList[i], image, i)

In [ ]:
#Reassemble drawn images here into video
images = []

for i in range(0, len(os.listdir('data')) - 1):
  images.append("data/%d.jpg" % i)

frame = cv2.imread('data/0.jpg')
height, width, layers = frame.shape

video = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 20.0, (width,height))

for image in images:
    video.write(cv2.imread(image))

cv2.destroyAllWindows()
video.release()